<a href="https://colab.research.google.com/github/momo54/large_scale_data_management/blob/main/Wikipedia_Events.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Streaming Wikipedia Events

FROM: https://github.com/donaghhorgan/COMP9033/blob/master/lectures/12%20-%20Streaming%20Data%20Analysis.pdf



In [1]:
#install software
# No clustering for this install !!
!pip install pyspark
!pip install -q findspark
import findspark
findspark.init()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 41 kB/s 
     |████████████████████████████████| 199 kB 39.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=0b2132bdf75c16dcdc62f58545e9a7a09deb4954bc21aeb2bf1cb53a23476bad
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [2]:
# start the spark session

from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

spark = SparkSession \
    .builder \
    .appName("Wikipedia Event") \
    .getOrCreate()

### Streaming Wikipedia events

Currently, Spark supports three kinds of streaming connection out of the box:

FROM:
https://github.com/donaghhorgan/COMP9033/blob/master/lectures/12%20-%20Streaming%20Data%20Analysis.pdf 

1. Connect to the Wikipedia RecentChanges stream using SSEClient.
2. Create a local socket connection on port 50000.
3. When a client (e.g. Spark) connects to the local socket, relay the next available event to it from the event stream.

In [3]:
!pip install sseclient
from pyspark.streaming import StreamingContext
from sseclient import SSEClient
import threading

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sseclient: filename=sseclient-0.0.27-py3-none-any.whl size=5584 sha256=fa879acbac7e9bcde15a5f6787a4db9c41d0f75ad4a051c8e685d587e106c568
  Stored in directory: /root/.cache/pip/wheels/07/67/7e/96edf627ac746de1a5c5cbb8d59ed960f033b8352dc12c545d
Successfully built sseclient


In [14]:
!rm test.json

msgs = SSEClient('https://stream.wikimedia.org/v2/stream/recentchange')

i=0
with open("test.json", "w") as outfile:
  for e in msgs:
    print(e)
    outfile.write(f"{e}\n")
    i=i+1
    if i>10:
      break

rm: cannot remove 'test.json': No such file or directory

{"$schema":"/mediawiki/recentchange/1.0.0","meta":{"uri":"https://commons.wikimedia.org/wiki/File:El_Paso_Times._(El_Paso,_Tex.),_Vol._Eighth_Year,_No._63,_Ed._1_Wednesday,_March_14,_1888_-_DPLA_-_c2ffe1944c348620511b33d4b48a14d1_(page_6).jpg","request_id":"4227c2c7-6bca-43a3-85a4-71cfda7a3ce5","id":"1cf38627-f9b7-4837-b16b-bb5b6d2cfaa8","dt":"2022-10-27T06:51:31Z","domain":"commons.wikimedia.org","stream":"mediawiki.recentchange","topic":"eqiad.mediawiki.recentchange","partition":0,"offset":4263841544},"id":2039185896,"type":"edit","namespace":6,"title":"File:El Paso Times. (El Paso, Tex.), Vol. Eighth Year, No. 63, Ed. 1 Wednesday, March 14, 1888 - DPLA - c2ffe1944c348620511b33d4b48a14d1 (page 6).jpg","comment":"/* wbeditentity-update:0| */ automatically modify [[Commons:Structured data|structured data]] based on file information: meta","timestamp":1666853491,"user":"SchlurcherBot","bot":true,"minor":false,"patrolled":true,"le

In [12]:
# simple thread that read wikipedia upddates and print it in files 
# with 100 update per file

!rm -rf streaming/*
!mkdir streaming

import uuid
import json

def read_wikipedia():
    msgs = SSEClient('https://stream.wikimedia.org/v2/stream/recentchange')

    buffer=[]
    i=0
    for e in msgs:
      buffer.append(e)
      if len(buffer)>100:
        with open(f"streaming/{str(uuid.uuid4())}.json", "w") as outfile:
          for m in buffer:
            outfile.write(f"{m}\n")
          outfile.close()
          #json.dump([ob.__dict__ for ob in buffer], outfile)
        buffer.clear()
        i=i+1
        if i>3:
          return

threading.Thread(target=read_wikipedia).start()
#read_wikipedia()


mkdir: cannot create directory ‘streaming’: File exists


In [16]:
# just extract the schema of a wikipedia update
# required to populate the input table of Structured Stream

import json

#don't forget the r for raw string...
data=r"""{"$schema":"/mediawiki/recentchange/1.0.0","meta":{"uri":"https://www.wikidata.org/wiki/Q114775139","request_id":"31f7d9b1-12c4-4de4-bee7-fbdd8c055c4f","id":"6580fd51-0bde-4f48-8186-8b1f202c4e14","dt":"2022-10-20T13:20:31Z","domain":"www.wikidata.org","stream":"mediawiki.recentchange","topic":"eqiad.mediawiki.recentchange","partition":0,"offset":4248495942},"id":1809138679,"type":"edit","namespace":0,"title":"Q114775139","comment":"/* wbsetclaim-create:2||1 */ [[Property:P6634]]: ellen-rudnick-364a793","timestamp":1666272031,"user":"Korimako","bot":false,"minor":false,"patrolled":true,"length":{"old":2211,"new":2575},"revision":{"old":1754930719,"new":1754930819},"server_url":"https://www.wikidata.org","server_name":"www.wikidata.org","server_script_path":"/w","wiki":"wikidatawiki","parsedcomment":"‎<span dir=\"auto\"><span class=\"autocomment\">Created claim: </span></span> <a href=\"/wiki/Property:P6634\" title=\"Property:P6634\">Property:P6634</a>: ellen-rudnick-364a793"}"""

#don't forget to build a list ie [data]...
rdd=spark.sparkContext.parallelize([data])
df=spark.read.json(rdd)
df.show()

df.printSchema()

#schema is computed ;)
df.schema


+--------------------+-----+--------------------+----------+------------+--------------------+-----+---------+--------------------+---------+--------------------+----------------+------------------+--------------------+----------+----------+----+--------+------------+
|             $schema|  bot|             comment|        id|      length|                meta|minor|namespace|       parsedcomment|patrolled|            revision|     server_name|server_script_path|          server_url| timestamp|     title|type|    user|        wiki|
+--------------------+-----+--------------------+----------+------------+--------------------+-----+---------+--------------------+---------+--------------------+----------------+------------------+--------------------+----------+----------+----+--------+------------+
|/mediawiki/recent...|false|/* wbsetclaim-cre...|1809138679|{2575, 2211}|{www.wikidata.org...|false|        0|‎<span dir="auto"...|     true|{1754930819, 1754...|www.wikidata.org|              

StructType([StructField('$schema', StringType(), True), StructField('bot', BooleanType(), True), StructField('comment', StringType(), True), StructField('id', LongType(), True), StructField('length', StructType([StructField('new', LongType(), True), StructField('old', LongType(), True)]), True), StructField('meta', StructType([StructField('domain', StringType(), True), StructField('dt', StringType(), True), StructField('id', StringType(), True), StructField('offset', LongType(), True), StructField('partition', LongType(), True), StructField('request_id', StringType(), True), StructField('stream', StringType(), True), StructField('topic', StringType(), True), StructField('uri', StringType(), True)]), True), StructField('minor', BooleanType(), True), StructField('namespace', LongType(), True), StructField('parsedcomment', StringType(), True), StructField('patrolled', BooleanType(), True), StructField('revision', StructType([StructField('new', LongType(), True), StructField('old', LongTyp

## Streaming analysis

Now that we have our stream relay set up, we can start to analyse its contents. First, let's initialise a [`SparkContext`](https://spark.apache.org/docs/2.1.0/api/python/pyspark.html#pyspark.SparkContext) object, which will represent our connection to the Spark cluster. To do this, we must first specify the URL of the master node to connect to. As we're only running this notebook for demonstration purposes, we can just run the cluster locally, as follows:

Next, we create a [`StreamingContext`](https://spark.apache.org/docs/latest/api/python/pyspark.streaming.html#pyspark.streaming.StreamingContext) object, which represents the streaming functionality of our Spark cluster. When we create the context, we must specify a batch duration time (in seconds), to tell Spark how often it should process data from the stream. Let's process the Wikipedia data in batches of one second:

In [17]:
# stopping all active queries
# you will understand ;-)
for q in spark.streams.active:
  print("stopping",q.name)
  q.stop()
spark.sql(f"drop table wikipedia")

AnalysisException: ignored

In [18]:
lines = spark \
    .readStream \
    .option("cleanSource","delete") \
    .format("json") \
    .schema(df.schema) \
    .load("streaming") \

results=lines.groupBy("user").count()

query = results \
    .writeStream \
    .outputMode("complete") \
    .format("memory") \
    .option("cleanSource","delete") \
    .queryName("wikipedia") \
    .start()

#query.awaitTermination()
print(query.isActive)

True


In [22]:
spark.sql("select * from wikipedia order by count DESC").show(3)
spark.sql("select count(*) from wikipedia").show()

#!cp test.json streaming

+--------------+-----+
|          user|count|
+--------------+-----+
|       AskeBot|   38|
|         Tulsi|   38|
|VladimirPF-Bot|   29|
+--------------+-----+
only showing top 3 rows

+--------+
|count(1)|
+--------+
|      98|
+--------+

